In [1]:
%matplotlib notebook
# Load data
import sys
sys.path.append('..')
from torch import nn
import torch.nn.functional as F
import numpy as np
import torch
from utils.data import fetch_data_with_label, label_to_int, divide_data, process_data_3d,process_batch_data_3d
import glob
import itertools

In [2]:
task = 'bin_picking'
if task == 'assembly':
        n_global_states = 4
        n_clusters = 5
        target_dir = '../../roboSim/data/assembly/*/*.pickle'
elif task == 'bin_picking':
        n_global_states = 2
        n_clusters = 2
        target_dir = '../../roboSim/data/bin_picking/*/*.pickle'
        
logfile_path = glob.glob(target_dir)
#coordinates and object kind for nut and bolt
data, label = fetch_data_with_label(logfile_path, n_global_states)
n_training = 2
n_val = 10
x_train, y_train, x_left, y_left= divide_data(data,label,n_training)
x_val, y_val, x_test, y_test = divide_data(x_left, y_left, n_val)
# Add noise and anchor information to the data
x_train_processed, y_train_processed = process_batch_data_3d(x_train, y_train, with_anchor = False, task = 'bin_picking')
x_val_processed, y_val_processed = process_batch_data_3d(x_val, y_val, with_anchor = False, task = 'bin_picking')
x_test_processed, y_test_processed = process_batch_data_3d(x_test, y_test, with_anchor = False, task = 'bin_picking')

x_train, y_train = torch.from_numpy(x_train_processed).float(), torch.from_numpy(y_train_processed.astype(int)).view(-1, 1).long()
x_val, y_val = torch.from_numpy(x_val_processed).float(), torch.from_numpy(y_val_processed.astype(int)).view(-1, 1).long()
x_test, y_test = torch.from_numpy(x_test_processed).float(), torch.from_numpy(y_test_processed.astype(int)).view(-1, 1).long()

input_dim = x_train_processed.shape[1]
output_dim = n_clusters


In [3]:
x_train

tensor([[ 0.3845,  0.0307, -0.2556,  0.9280,  0.0720],
        [ 0.3733,  0.0344, -0.1080,  0.8646,  0.1354],
        [ 0.4397,  0.0558,  0.0043,  0.0733,  0.9267],
        [ 0.5471,  0.0625, -0.1496,  0.1536,  0.8464],
        [ 0.3967,  0.0198, -0.2619,  0.8728,  0.1272],
        [ 0.3769,  0.0307, -0.1017,  0.9854,  0.0146],
        [-0.4851,  0.0565, -0.5864,  0.1097,  0.8903],
        [ 0.4567,  0.0562, -0.2088,  0.1378,  0.8622],
        [-0.4851,  0.0565, -0.5864,  0.1097,  0.8903],
        [-0.4851,  0.0565, -0.5864,  0.1097,  0.8903],
        [-0.4851,  0.0565, -0.5864,  0.1097,  0.8903],
        [-0.4851,  0.0565, -0.5864,  0.1097,  0.8903],
        [-0.4851,  0.0565, -0.5864,  0.1097,  0.8903],
        [-0.4851,  0.0565, -0.5864,  0.1097,  0.8903],
        [ 0.4100,  0.0426,  0.1051,  0.8972,  0.1028],
        [ 0.2907,  0.0363, -0.1749,  0.9117,  0.0883],
        [ 0.4573,  0.0673, -0.1695,  0.1372,  0.8628],
        [ 0.3426,  0.0591, -0.1070,  0.0754,  0.9246],
        [-

In [13]:
len(y_val)

10

In [6]:
class MyNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, output_dim)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x